In [1]:
import PyPDF2
import os
import pandas as pd
from natsort import natsorted

In [2]:
input_pdf_path = 'input_pdf/LOAD SCHEDULE - MDB (1).pdf'
output_directory = r"output/pdf/"

In [3]:
os.makedirs(output_directory, exist_ok=True)

In [4]:
with open(input_pdf_path, 'rb') as file:
    pdf_reader = PyPDF2.PdfReader(file)
    for page_num in range(21):
        pdf_writer = PyPDF2.PdfWriter()
        pdf_writer.add_page(pdf_reader.pages[page_num])

        output_path = os.path.join(output_directory, f'page_{page_num + 1}.pdf')

        with open(output_path, 'wb') as output_file:
            pdf_writer.write(output_file)
print('Extraction of pdf completed!')

Extraction of pdf completed!


In [5]:
pdf_directory = r'output/pdf/'
excel_directory = r'output/excel/'
data = []

In [6]:
import aspose.pdf as ap

for filename in os.listdir(pdf_directory):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_directory, filename)
        document = ap.Document(pdf_path)
        excel_filename = f'{os.path.splitext(filename)[0]}.xlsx'
        excel_path = os.path.join(excel_directory, excel_filename)
        save_option = ap.ExcelSaveOptions()
        document.save(excel_path, save_option)
print('Conversion to Excel complete!')

Conversion to Excel complete!


In [7]:
df1 = pd.read_excel(r'output/excel/page_1.xlsx')

In [8]:
sliced_dfs = {}
excel_files = natsorted([f for f in os.listdir(excel_directory) if f.endswith('.xlsx')])

In [9]:
for idx, filename in enumerate(excel_files):
    excel_path = os.path.join(excel_directory, filename)
    
    # Load the Excel file into a DataFrame
    df = pd.read_excel(excel_path)
    
    # Slice the DataFrame (rows 4 to 30)
    sliced_df = df.iloc[0:33]
    
    # Store the sliced DataFrame in the dictionary
    key = f'df{idx + 1}'
    sliced_dfs[key] = sliced_df

    # Save the sliced DataFrame to a CSV file
    csv_filename = f'{key}.csv'
    csv_path = os.path.join(excel_directory, csv_filename)
    sliced_dfs[key].to_csv(csv_path, index=False)

print('Slicing, storing, and saving DataFrames complete!')

Slicing, storing, and saving DataFrames complete!


In [10]:
total_nan_rows = 0
for key, sliced_df in sliced_dfs.items():
    nan_rows_count = sliced_df.isna().all(axis=1).sum()
    total_nan_rows += nan_rows_count
print(f'Total number of rows with only NaN values: {total_nan_rows}')

Total number of rows with only NaN values: 91


In [11]:
cleaned_dfs = {}

In [12]:
for idx, (key, df) in enumerate(sliced_dfs.items(), start=1):
    # Ensure working with a copy to avoid warnings
    df = df.copy()
    
    # Drop rows where all elements are NaN
    cleaned_df = df.dropna(how='all').copy()
    
    # Dynamically create a key like df1, df2, etc.
    cleaned_dfs[f'df{idx}'] = cleaned_df
print('Rows with only NaN values have been removed from all DataFrames.')

Rows with only NaN values have been removed from all DataFrames.


In [13]:
for key, df in cleaned_dfs.items():
    # Check if the DataFrame has at least five columns
    if df.shape[1] >= 6:
        # Rename the columns as specified
        df.columns = [
            df.columns[0],  # Keep the first column name unchanged
            'POLE',
            'FS/IS', # Rename the third column to 'ACB'
            'ACB/MCCB',         # Rename the fourth column to 'MCCB'
            'MCB',          # Rename the fifth column to 'MCB'
            'FAULT DUTY'    # Rename the sixth column to 'FAULT DUTY'
        ] + list(df.columns[6:])  # Keep the remaining columns unchanged

        # Keep only the first 8 columns
        df = df.iloc[:, :8]
        
        cleaned_dfs[key] = df

print('Columns have been renamed and trimmed to the first 8 columns in all DataFrames.')

Columns have been renamed and trimmed to the first 8 columns in all DataFrames.


In [14]:
cleaned_dfs['df7']

,Evaluation Only. Created with Aspose.PDF. Copyright 2002-2024 Aspose Pty Ltd.,POLE,FS/IS,ACB/MCCB,MCB,FAULT DUTY,Unnamed: 6,Unnamed: 7
0,PAGE - SCH - 6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IAN BANHAM & ASSOCIATES,ELECTRICAL,NaN,NaN,NaN,NaN,NaN,NaN
2,PROJECT :,B+G+M+1 FLOOR CAR SHOWROOM & OFFICE BUILDING D...,NaN,NaN,NaN,NaN,NaN,NaN
3,SMDB REF :,SMDB - BN,NaN,NaN,NaN,NaN,NaN,CONSULTANT :
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SR.,CIRCUIT,SP/,RATING - AMPS,NaN,NaN,FAULT,PVC/XLPE
6,NO,FEEDER\nDB NO.,TP,ACB,MCCB,MCB,DUTY\nKA,SWA/PVC/SC
7,NaN,INCOMER,TP,NaN,500\n(N/A),NaN,30,XLPE/SWA/PVC
8,NaN,OUTGOING,NaN,NaN,NaN,NaN,NaN,NaN
9,1,DB-BN1,TP,NaN,40,NaN,30,XLPE/SWA/PVC


In [15]:
# for key, df in cleaned_dfs.items():
#     df = df.dropna(thresh=df.shape[1] - 7)
#     cleaned_dfs[key] = df

# print('deleted row more that 3 nan values')

In [16]:
cleaned_dfs['df7']

,Evaluation Only. Created with Aspose.PDF. Copyright 2002-2024 Aspose Pty Ltd.,POLE,FS/IS,ACB/MCCB,MCB,FAULT DUTY,Unnamed: 6,Unnamed: 7
0,PAGE - SCH - 6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IAN BANHAM & ASSOCIATES,ELECTRICAL,NaN,NaN,NaN,NaN,NaN,NaN
2,PROJECT :,B+G+M+1 FLOOR CAR SHOWROOM & OFFICE BUILDING D...,NaN,NaN,NaN,NaN,NaN,NaN
3,SMDB REF :,SMDB - BN,NaN,NaN,NaN,NaN,NaN,CONSULTANT :
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SR.,CIRCUIT,SP/,RATING - AMPS,NaN,NaN,FAULT,PVC/XLPE
6,NO,FEEDER\nDB NO.,TP,ACB,MCCB,MCB,DUTY\nKA,SWA/PVC/SC
7,NaN,INCOMER,TP,NaN,500\n(N/A),NaN,30,XLPE/SWA/PVC
8,NaN,OUTGOING,NaN,NaN,NaN,NaN,NaN,NaN
9,1,DB-BN1,TP,NaN,40,NaN,30,XLPE/SWA/PVC


In [17]:
combined_df = pd.concat(cleaned_dfs.values(), ignore_index=True)

# Count occurrences of each product in the first column
product_counts = combined_df.iloc[:, 0].value_counts().to_dict()

# Add the count column to each DataFrame in cleaned_dfs
for key, df in cleaned_dfs.items():
    # Use .loc[] to avoid SettingWithCopyWarning
    df['Count'] = df.iloc[:, 0].map(product_counts).fillna(0).astype(int)

    # Update the DataFrame in the dictionary
    cleaned_dfs[key] = df

print('Count column has been added to each DataFrame in cleaned_dfs.')

Count column has been added to each DataFrame in cleaned_dfs.


C:\Users\USER\AppData\Local\Temp\ipykernel_14656\1023208774.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Count'] = df.iloc[:, 0].map(product_counts).fillna(0).astype(int)


In [18]:
cleaned_dfs['df17']

,Evaluation Only. Created with Aspose.PDF. Copyright 2002-2024 Aspose Pty Ltd.,POLE,FS/IS,ACB/MCCB,MCB,FAULT DUTY,Unnamed: 6,Unnamed: 7,Count
0,PAGE - SCH - 16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,IAN BANHAM & ASSOCIATES,ELECTRICAL,NaN,NaN,NaN,NaN,NaN,NaN,20
2,PROJECT :,NaN,B+G+M+1 FLOOR CAR SHOWROOM & OFFICE BUILDING D...,NaN,NaN,NaN,NaN,NaN,17
3,SMDBREF :,NaN,SMDB-GN2B,NaN,NaN,NaN,NaN,CONSULTANT :,8
4,SR.,CIRCUIT,SP/,RATING - AMPS,NaN,NaN,FAULT,PVC/XLPE,20
5,NO,FEEDER\nDB NO.,TP,ACB,MCCB,MCB,DUTY\nKA,SWA/PVC/SC,20
6,NaN,INCOMER,TP,NaN,500\n(N/A),NaN,35,XLPE/SWA/PVC,0
7,NaN,OUTGOING,NaN,NaN,NaN,NaN,NaN,NaN,0
8,1,BAY AREA,TP,NaN,63,NaN,35,XLPE/SWA/PVC,18
9,2,BAY AREA,TP,NaN,63,NaN,35,XLPE/SWA/PVC,17


In [19]:
combined_df = pd.concat(cleaned_dfs.values(), ignore_index=True)

# Count occurrences of each product in the first column
product_counts = combined_df.iloc[:, 0].value_counts().to_dict()

# Add the count column to each DataFrame in cleaned_dfs
for key, df in cleaned_dfs.items():
    # Ensure we're working with a copy of the DataFrame to avoid SettingWithCopyWarning
    df_copy = df.copy()
    
    # Add the count column
    df_copy.loc[:, 'Count'] = df_copy.iloc[:, 0].map(product_counts).fillna(0).astype(int)
    
    # Update the DataFrame in the dictionary
    cleaned_dfs[key] = df_copy

print('Count column has been added as integers to all DataFrames in cleaned_dfs.')

Count column has been added as integers to all DataFrames in cleaned_dfs.


In [20]:
cleaned_dfs['df10']

,Evaluation Only. Created with Aspose.PDF. Copyright 2002-2024 Aspose Pty Ltd.,POLE,FS/IS,ACB/MCCB,MCB,FAULT DUTY,Unnamed: 6,Unnamed: 7,Count
0,PAGE - SCH - 9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,IAN BANHAM & ASSOCIATES,ELECTRICAL,NaN,NaN,NaN,NaN,NaN,NaN,20
2,PROJECT :,B+G+M+1 FLOOR CAR SHOWROOM & OFFICE BUILDING D...,NaN,NaN,NaN,NaN,NaN,NaN,17
3,SMDB REF :,SMDB - BE2,NaN,NaN,NaN,NaN,NaN,CONSULTANT :,7
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,SR.,CIRCUIT,SP/,RATING - AMPS,NaN,NaN,FAULT,PVC/XLPE,20
6,NO,FEEDER\nDB NO.,TP,ACB,MCCB,MCB,DUTY\nKA,SWA/PVC/SC,20
7,NaN,INCOMER,TP,NaN,125\n(N/A),NaN,30,FIRE PROOF,0
8,NaN,OUTGOING,NaN,NaN,NaN,NaN,NaN,NaN,0
9,1,DB-BE2,TP,NaN,40,NaN,30,FIRE PROOF,18


In [33]:
lv_dfs = []
smdb_dfs = []
db_dfs = []
other_dfs = []

# Define the pattern for identifying rows with "REF"
ref_pattern = "REF"

# Define a function to preprocess DataFrames
def preprocess_dataframe(df):
    # Keep rows where the first column contains "REF"
    ref_rows = df[df.iloc[:, 0].astype(str).str.contains(ref_pattern, na=False)]
    
    # Drop rows with more than 6 NaN values, except for rows with "REF"
    df_filtered = df[~df.index.isin(ref_rows.index)]  # Exclude REF rows from deletion
    df_filtered = df_filtered[df_filtered.isna().sum(axis=1) <= 5]
    
    # Concatenate REF rows back to the filtered DataFrame
    df_preprocessed = pd.concat([df_filtered, ref_rows]).sort_index()
    return df_preprocessed

# Define a function to categorize DataFrames
def categorize_dataframe(df):
    # Limit checking to the first 7 rows
    rows_to_check = df.head(7)
    
    # Check if any row in the first 7 rows contains "REF" in the first column
    for _, row in rows_to_check.iterrows():
        if ref_pattern in str(row.iloc[0]):
            # Check the values in the first 3 columns of the matched row
            first_column_value = str(row.iloc[0]).strip()
            second_column_value = str(row.iloc[1]).strip() if df.shape[1] > 1 else ""
            third_column_value = str(row.iloc[2]).strip() if df.shape[1] > 2 else ""
            
            # Categorize based on the content of the columns
            if "LV" in first_column_value or "LV" in second_column_value or "LV" in third_column_value:
                return 'LV'
            elif "SMDB" in first_column_value or "SMDB" in second_column_value or "SMDB" in third_column_value:
                return 'SMDB'
            elif "DB" in first_column_value or "DB" in second_column_value or "DB" in third_column_value:
                return 'DB'
            else:
                return 'Other'
    return 'Other'

# Iterate over each DataFrame in cleaned_dfs
for key, df in cleaned_dfs.items():
    print(f"Processing DataFrame: {key}")
    
    # Preprocess the DataFrame
    df = preprocess_dataframe(df)
    
    # Categorize the DataFrame
    category = categorize_dataframe(df)
    
    # Categorize DataFrames based on the determined category
    if category == 'LV':
        lv_dfs.append(df)
    elif category == 'SMDB':
        smdb_dfs.append(df)
    elif category == 'DB':
        db_dfs.append(df)
    else:
        other_dfs.append(df)

# Concatenate DataFrames for each category
lv_df = pd.concat(lv_dfs, ignore_index=True) if lv_dfs else pd.DataFrame()
smdb_df = pd.concat(smdb_dfs, ignore_index=True) if smdb_dfs else pd.DataFrame()
db_df = pd.concat(db_dfs, ignore_index=True) if db_dfs else pd.DataFrame()
other_df = pd.concat(other_dfs, ignore_index=True) if other_dfs else pd.DataFrame()

print('DataFrames have been categorized and combined into LV, SMDB, DB, and Others.')

# Optionally, check the lengths of the lists to ensure they contain DataFrames
print(f"Number of DataFrames in LV: {len(lv_dfs)}")
print(f"Number of DataFrames in SMDB: {len(smdb_dfs)}")
print(f"Number of DataFrames in DB: {len(db_dfs)}")
print(f"Number of DataFrames in Others: {len(other_dfs)}")

Processing DataFrame: df1
Processing DataFrame: df2
Processing DataFrame: df3
Processing DataFrame: df4
Processing DataFrame: df5
Processing DataFrame: df6
Processing DataFrame: df7
Processing DataFrame: df8
Processing DataFrame: df9
Processing DataFrame: df10
Processing DataFrame: df11
Processing DataFrame: df12
Processing DataFrame: df13
Processing DataFrame: df14
Processing DataFrame: df15
Processing DataFrame: df16
Processing DataFrame: df17
Processing DataFrame: df18
Processing DataFrame: df19
Processing DataFrame: df20
Processing DataFrame: df21
DataFrames have been categorized and combined into LV, SMDB, DB, and Others.
Number of DataFrames in LV: 2
Number of DataFrames in SMDB: 15
Number of DataFrames in DB: 1
Number of DataFrames in Others: 3


In [34]:
from IPython.display import display, HTML
import os

file_path = 'categorized_dataframes.xlsx'

# Create an Excel writer object using openpyxl
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    # Ensure at least one DataFrame is written to ensure visibility
    sheets_written = False

    if not lv_df.empty:
        lv_df.to_excel(writer, sheet_name='COST LV', index=False)
        sheets_written = True
    if not smdb_df.empty:
        smdb_df.to_excel(writer, sheet_name='COST SMDB', index=False)
        sheets_written = True
    if not db_df.empty:
        db_df.to_excel(writer, sheet_name='COST DB', index=False)
        sheets_written = True
    if not other_df.empty:
        other_df.to_excel(writer, sheet_name='Others', index=False)
        sheets_written = True

    # Check if any sheets were actually written
    if not sheets_written:
        raise ValueError("No DataFrames were written to the Excel file. Ensure at least one DataFrame has data.")

# Check if the file exists and is not empty
if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
    # Create a download link
    download_link = f'<a href="{file_path}" download>Click here to download the Excel file</a>'
    display(HTML(download_link))
else:
    print('File creation failed or file is empty.')

print(f'DataFrames have been saved to {file_path}.')

DataFrames have been saved to categorized_dataframes.xlsx.


In [42]:
smdb_dfs[10]

,Evaluation Only. Created with Aspose.PDF. Copyright 2002-2024 Aspose Pty Ltd.,POLE,FS/IS,ACB/MCCB,MCB,FAULT DUTY,Unnamed: 6,Unnamed: 7,Count
3,SMDBREF :,NaN,SMDB-GN2B,NaN,NaN,NaN,NaN,CONSULTANT :,8
4,SR.,CIRCUIT,SP/,RATING - AMPS,NaN,NaN,FAULT,PVC/XLPE,20
5,NO,FEEDER\nDB NO.,TP,ACB,MCCB,MCB,DUTY\nKA,SWA/PVC/SC,20
6,NaN,INCOMER,TP,NaN,500\n(N/A),NaN,35,XLPE/SWA/PVC,0
8,1,BAY AREA,TP,NaN,63,NaN,35,XLPE/SWA/PVC,18
9,2,BAY AREA,TP,NaN,63,NaN,35,XLPE/SWA/PVC,17
10,3,BAY AREA,TP,NaN,63,NaN,35,XLPE/SWA/PVC,14
11,4,BAY AREA,TP,NaN,63,NaN,35,XLPE/SWA/PVC,16
12,5,BAY AREA,TP,NaN,63,NaN,35,XLPE/SWA/PVC,12
13,6,BAY AREA,TP,NaN,63,NaN,35,XLPE/SWA/PVC,10
